# Démonstration MLOps - Tracking, Versioning et CI/CD

Ce notebook documente la mise en œuvre de la démarche MLOps complète pour le projet Air Paradis.

---

## Table des matières

1. [Introduction aux Principes MLOps](#1-intro)
2. [Tracking des Expérimentations avec MLFlow](#2-mlflow)
3. [Stockage et Versioning des Modèles](#3-versioning)
4. [Gestion de Version avec Git](#4-git)
5. [Tests Unitaires](#5-tests)
6. [Pipeline CI/CD avec GitHub Actions](#6-cicd)
7. [Monitoring en Production](#7-monitoring)
8. [Conclusion](#8-conclusion)

## 1. Introduction aux Principes MLOps <a id="1-intro"></a>

### Qu'est-ce que le MLOps ?

**MLOps** (Machine Learning Operations) est l'ensemble des pratiques qui vise à déployer et maintenir des modèles de Machine Learning en production de manière fiable et efficace.

### Principes Clés

1. **Reproductibilité** : Garantir que les expérimentations peuvent être reproduites à l'identique
2. **Traçabilité** : Suivre toutes les versions de code, données et modèles
3. **Automatisation** : Automatiser les tests, le déploiement et le monitoring
4. **Collaboration** : Faciliter le travail en équipe (Data Scientists, ML Engineers, DevOps)
5. **Qualité** : Garantir la qualité du code et des modèles via tests automatisés
6. **Monitoring** : Surveiller les performances en production et détecter les dérives

### Architecture MLOps du Projet

```
┌─────────────────────────────────────────────────────────────────┐
│                     DÉVELOPPEMENT                                │
│  ┌──────────────┐  ┌──────────────┐  ┌──────────────┐          │
│  │   Notebooks  │→ │    MLFlow    │→ │  Git/GitHub  │          │
│  │ Experiments  │  │   Tracking   │  │   Versioning │          │
│  └──────────────┘  └──────────────┘  └──────────────┘          │
└─────────────────────────────────────────────────────────────────┘
                            ↓
┌─────────────────────────────────────────────────────────────────┐
│                   CI/CD PIPELINE                                 │
│  ┌──────────────┐  ┌──────────────┐  ┌──────────────┐          │
│  │GitHub Actions│→ │ Unit Tests   │→ │  Docker Build│          │
│  │   Trigger    │  │   (pytest)   │  │ & Push       │          │
│  └──────────────┘  └──────────────┘  └──────────────┘          │
└─────────────────────────────────────────────────────────────────┘
                            ↓
┌─────────────────────────────────────────────────────────────────┐
│                    PRODUCTION                                    │
│  ┌──────────────┐  ┌──────────────┐  ┌──────────────┐          │
│  │ Azure/Heroku │→ │  Flask API   │→ │  App Insights│          │
│  │   Deployment │  │  Inference   │  │  Monitoring  │          │
│  └──────────────┘  └──────────────┘  └──────────────┘          │
└─────────────────────────────────────────────────────────────────┘
```

In [1]:
# Import des bibliothèques
import mlflow
import mlflow.sklearn
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Configuration
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✓ Bibliothèques importées")

✓ Bibliothèques importées


## 2. Tracking des Expérimentations avec MLFlow <a id="2-mlflow"></a>

### 2.1 Qu'est-ce que MLFlow ?

**MLFlow** est une plateforme open-source pour gérer le cycle de vie complet du Machine Learning :
- **MLFlow Tracking** : Enregistrer les paramètres, métriques et artefacts
- **MLFlow Models** : Gérer et déployer les modèles
- **MLFlow Model Registry** : Versionner et gérer les modèles en production

### 2.2 Configuration de MLFlow

Dans nos notebooks, nous avons configuré MLFlow pour tracker toutes les expérimentations :

In [2]:
# Configuration de MLFlow
mlflow.set_tracking_uri("file:///home/thomas/mlruns")  # URI local (ou serveur distant)
mlflow.set_experiment("sentiment-analysis-twitter")

print(f"MLFlow Tracking URI: {mlflow.get_tracking_uri()}")
print(f"Experiment actif: {mlflow.get_experiment_by_name('sentiment-analysis-twitter').name}")

MLFlow Tracking URI: file:///home/thomas/mlruns
Experiment actif: sentiment-analysis-twitter


### 2.3 Exemple de Tracking d'un Run

Voici comment nous avons tracké chaque expérimentation dans nos notebooks :

In [3]:
# Exemple de code utilisé dans les notebooks pour tracker un modèle
example_code = '''
with mlflow.start_run(run_name="logistic-regression-tfidf"):
    # Log des paramètres
    mlflow.log_param("model_type", "LogisticRegression")
    mlflow.log_param("preprocessing", "lemmatization")
    mlflow.log_param("vectorizer", "TfidfVectorizer")
    mlflow.log_param("max_features", 10000)
    mlflow.log_param("C", 1.0)
    
    # Entraînement du modèle
    model = LogisticRegression(C=1.0, max_iter=1000)
    model.fit(X_train_tfidf, y_train)
    
    # Prédictions et métriques
    y_pred = model.predict(X_test_tfidf)
    y_proba = model.predict_proba(X_test_tfidf)[:, 1]
    
    # Log des métriques
    mlflow.log_metric("accuracy", accuracy_score(y_test, y_pred))
    mlflow.log_metric("f1_score", f1_score(y_test, y_pred))
    mlflow.log_metric("roc_auc", roc_auc_score(y_test, y_proba))
    mlflow.log_metric("precision", precision_score(y_test, y_pred))
    mlflow.log_metric("recall", recall_score(y_test, y_pred))
    
    # Log du modèle
    mlflow.sklearn.log_model(model, "model")
    
    # Log des artefacts (graphiques, matrices de confusion)
    plt.figure(figsize=(8, 6))
    cm = confusion_matrix(y_test, y_pred)
    sns.heatmap(cm, annot=True, fmt='d')
    plt.savefig('confusion_matrix.png')
    mlflow.log_artifact('confusion_matrix.png')
'''

print("Exemple de code de tracking MLFlow :")
print("="*80)
print(example_code)
print("="*80)

Exemple de code de tracking MLFlow :

with mlflow.start_run(run_name="logistic-regression-tfidf"):
    # Log des paramètres
    mlflow.log_param("model_type", "LogisticRegression")
    mlflow.log_param("preprocessing", "lemmatization")
    mlflow.log_param("vectorizer", "TfidfVectorizer")
    mlflow.log_param("max_features", 10000)
    mlflow.log_param("C", 1.0)
    
    # Entraînement du modèle
    model = LogisticRegression(C=1.0, max_iter=1000)
    model.fit(X_train_tfidf, y_train)
    
    # Prédictions et métriques
    y_pred = model.predict(X_test_tfidf)
    y_proba = model.predict_proba(X_test_tfidf)[:, 1]
    
    # Log des métriques
    mlflow.log_metric("accuracy", accuracy_score(y_test, y_pred))
    mlflow.log_metric("f1_score", f1_score(y_test, y_pred))
    mlflow.log_metric("roc_auc", roc_auc_score(y_test, y_proba))
    mlflow.log_metric("precision", precision_score(y_test, y_pred))
    mlflow.log_metric("recall", recall_score(y_test, y_pred))
    
    # Log du modèle
    

### 2.4 Visualisation des Runs dans MLFlow UI

Pour visualiser les expérimentations, lancer MLFlow UI :

```bash
mlflow ui --backend-store-uri file:///home/thomas/mlruns
```

L'interface sera disponible sur `http://localhost:5000`

**Captures d'écran à inclure** :
- Vue de la liste des runs avec toutes les métriques
- Comparaison de plusieurs runs côte à côte
- Visualisation d'un run individuel avec paramètres et artefacts

In [4]:
# Récupération programmatique des runs MLFlow
from mlflow.tracking import MlflowClient

client = MlflowClient()
experiment = client.get_experiment_by_name("sentiment-analysis-twitter")

if experiment:
    runs = client.search_runs(
        experiment_ids=[experiment.experiment_id],
        order_by=["metrics.f1_score DESC"],
        max_results=10
    )
    
    print("\n" + "="*100)
    print("RUNS MLFLOW ENREGISTRÉS (Top 10 par F1-Score)")
    print("="*100)
    
    runs_data = []
    for run in runs:
        runs_data.append({
            'Run ID': run.info.run_id[:8],
            'Model': run.data.params.get('model_type', 'N/A'),
            'Preprocessing': run.data.params.get('preprocessing', 'N/A'),
            'F1-Score': f"{run.data.metrics.get('f1_score', 0):.4f}",
            'Accuracy': f"{run.data.metrics.get('accuracy', 0):.4f}",
            'ROC-AUC': f"{run.data.metrics.get('roc_auc', 0):.4f}",
            'Date': run.info.start_time
        })
    
    df_runs = pd.DataFrame(runs_data)
    print(df_runs.to_string(index=False))
    print("="*100)
    print(f"\nTotal runs enregistrés : {len(runs)}")
else:
    print("⚠️ Experiment 'sentiment-analysis-twitter' non trouvé")
    print("   Les runs seront créés lors de l'exécution des notebooks de modélisation")


RUNS MLFLOW ENREGISTRÉS (Top 10 par F1-Score)
  Run ID Model Preprocessing F1-Score Accuracy ROC-AUC          Date
6ed6b866   N/A lemmatization   0.0000   0.0000  0.0000 1767290175909
bc350ec2   N/A Lemmatization   0.0000   0.0000  0.0000 1766977882182
7a7c580e   N/A Lemmatization   0.0000   0.0000  0.0000 1766972672796
f4e09554   N/A      Stemming   0.0000   0.0000  0.0000 1766967560870
54aecd1c   N/A Lemmatization   0.0000   0.0000  0.0000 1766962850056
51ba5d83   N/A Lemmatization   0.0000   0.0000  0.0000 1766914867879
83d935b3   N/A Lemmatization   0.0000   0.0000  0.0000 1766914184369
af9429a3   N/A Lemmatization   0.0000   0.0000  0.0000 1766913904592
17613af1   N/A Lemmatization   0.0000   0.0000  0.0000 1766896055309
b49831ba   N/A Lemmatization   0.0000   0.0000  0.0000 1766891485037

Total runs enregistrés : 10


### 2.5 Avantages du Tracking MLFlow

✅ **Reproductibilité** : Tous les paramètres et configurations sont enregistrés

✅ **Comparaison facile** : Comparer visuellement les performances de dizaines de modèles

✅ **Historique complet** : Retrouver n'importe quelle expérimentation passée

✅ **Collaboration** : Partager les résultats avec l'équipe

✅ **Décisions éclairées** : Choisir le meilleur modèle basé sur des données objectives

## 3. Stockage et Versioning des Modèles <a id="3-versioning"></a>

### 3.1 MLFlow Model Registry

Le **Model Registry** permet de :
- Enregistrer les modèles avec des versions
- Gérer le cycle de vie (Staging, Production, Archived)
- Tracker qui a déployé quel modèle et quand
- Faciliter les rollbacks en cas de problème

### 3.2 Enregistrement d'un Modèle

In [5]:
# Exemple d'enregistrement d'un modèle dans le registry
example_registry = '''
# Après avoir entraîné et logué un modèle dans un run
with mlflow.start_run(run_name="best-logistic-regression") as run:
    # ... entraînement et logging ...
    
    # Enregistrer le modèle dans le registry
    model_uri = f"runs:/{run.info.run_id}/model"
    mv = mlflow.register_model(model_uri, "sentiment-classifier")
    
    print(f"Modèle enregistré: {mv.name}, version {mv.version}")

# Promouvoir un modèle en Production
client = MlflowClient()
client.transition_model_version_stage(
    name="sentiment-classifier",
    version=1,
    stage="Production"
)
'''

print("Exemple d'enregistrement dans Model Registry :")
print("="*80)
print(example_registry)
print("="*80)

Exemple d'enregistrement dans Model Registry :

# Après avoir entraîné et logué un modèle dans un run
with mlflow.start_run(run_name="best-logistic-regression") as run:
    # ... entraînement et logging ...
    
    # Enregistrer le modèle dans le registry
    model_uri = f"runs:/{run.info.run_id}/model"
    mv = mlflow.register_model(model_uri, "sentiment-classifier")
    
    print(f"Modèle enregistré: {mv.name}, version {mv.version}")

# Promouvoir un modèle en Production
client = MlflowClient()
client.transition_model_version_stage(
    name="sentiment-classifier",
    version=1,
    stage="Production"
)



### 3.3 Chargement d'un Modèle depuis le Registry

In [6]:
# Exemple de chargement d'un modèle depuis le registry
example_load = '''
import mlflow.sklearn

# Charger la dernière version en Production
model_name = "sentiment-classifier"
model_version_uri = f"models:/{model_name}/Production"
model = mlflow.sklearn.load_model(model_version_uri)

# Utiliser le modèle pour prédire
prediction = model.predict(["This is a great product!"])
print(f"Prediction: {prediction}")
'''

print("Exemple de chargement depuis Model Registry :")
print("="*80)
print(example_load)
print("="*80)

Exemple de chargement depuis Model Registry :

import mlflow.sklearn

# Charger la dernière version en Production
model_name = "sentiment-classifier"
model_version_uri = f"models:/{model_name}/Production"
model = mlflow.sklearn.load_model(model_version_uri)

# Utiliser le modèle pour prédire
prediction = model.predict(["This is a great product!"])
print(f"Prediction: {prediction}")



### 3.4 Cycle de Vie des Modèles

```
Development → Staging → Production → Archived
    ↓            ↓          ↓            ↓
  Tests     Validation  Déployé    Déprécié
```

**Workflow typique** :
1. Entraîner et logger un nouveau modèle
2. L'enregistrer dans le registry (état = None)
3. Le tester → si OK, promouvoir en **Staging**
4. Valider en Staging → si OK, promouvoir en **Production**
5. Ancien modèle → archiver
6. En cas de problème → rollback vers version précédente

## 4. Gestion de Version avec Git <a id="4-git"></a>

### 4.1 Structure du Repository

```
openclassrooms-projet7/
├── .github/
│   └── workflows/
│       └── deploy.yml          # CI/CD pipeline
├── api/
│   ├── app.py                  # Flask API
│   ├── requirements.txt        # Dépendances API
│   └── Dockerfile              # Container
├── data/
│   ├── raw/                    # Données brutes
│   └── processed/              # Données traitées
├── notebooks/
│   ├── 01_exploration.ipynb
│   ├── 02_preprocessing.ipynb
│   ├── 03_modele_simple.ipynb
│   ├── 04_modeles_avances.ipynb
│   └── 05_modele_bert.ipynb
├── models/
│   └── saved_models/           # Modèles sérialisés
├── tests/
│   ├── test_api.py             # Tests unitaires API
│   └── test_preprocessing.py   # Tests prétraitement
├── livrables/
│   ├── 01_comparaison_finale_modeles.ipynb
│   ├── 02_test_api_streamlit.ipynb
│   └── 03_mlops_demonstration.ipynb
├── streamlit_app.py            # Interface Streamlit
├── requirements.txt            # Dépendances projet
├── README.md                   # Documentation
└── .gitignore                  # Fichiers ignorés
```

### 4.2 Historique Git

Le projet contient au moins 3 versions distinctes accessibles via Git :

In [7]:
%%bash
# Afficher l'historique des commits
cd /mnt/c/Users/Thomas/Documents/Code/openclassrooms/openclassrooms-projet7
git log --oneline --graph --all -20

* 936761d test
* 4a066b5 test
* de4d81b Add demo tweets for presentation
* 7ccd9d9 Add final deliverables: blog article, presentation plan, and checklist
* 4e10814 Remove API keys from documentation - use empty strings
* b27bc26 Replace Azure Application Insights with PostHog Analytics
* a2e12bd Add Heroku API URL to Streamlit configuration
* c2c5669 Separate Streamlit interface into dedicated folder
* f1cfd00 Add requirements.txt at root for Streamlit Cloud
* 1fd5877 Fix Streamlit requirements versions for deployment
* 6b9fe0b improve api
* 5d40654 livrables
* c74dd68 bert 20 epoch
* 7cb1c3a Update confusion matrix and training history visualizations for BERT model
* 86a1313 notebook 5 fix
* 3658bd5 notebook 5
* 7c562b5 Clean up API directory
* 382766c Update scikit-learn to 1.7.2 to match vectorizer version
* ffde591 Add vectorizer idf_ attribute check on load
* 5afc9bf Force Heroku rebuild to load new vectorizer


### 4.3 Fichiers Clés de Versioning

#### requirements.txt

Liste de tous les packages avec versions exactes pour garantir la reproductibilité :

In [8]:
# Afficher le fichier requirements.txt
with open('../requirements.txt', 'r') as f:
    requirements = f.read()

print("Fichier requirements.txt :")
print("="*80)
print(requirements)
print("="*80)

Fichier requirements.txt :
# Streamlit Application Requirements

# Streamlit
streamlit>=1.28.0

# Data Processing
pandas>=2.0.0
numpy>=1.24.0,<2.0.0

# Visualization
plotly>=5.0.0

# API Calls
requests>=2.31.0

# Utilities
python-dotenv>=1.0.0

# Azure Application Insights (optionnel - pour monitoring)
opencensus-ext-azure>=1.1.0



#### .gitignore

Fichiers à ignorer (données volumineuses, secrets, cache) :

In [9]:
# Afficher le fichier .gitignore
import os
gitignore_path = '../.gitignore'

if os.path.exists(gitignore_path):
    with open(gitignore_path, 'r') as f:
        gitignore = f.read()
    
    print("Fichier .gitignore :")
    print("="*80)
    print(gitignore)
    print("="*80)
else:
    print("⚠️ Fichier .gitignore non trouvé")

Fichier .gitignore :
.venv/

data/

# Fichiers de modèles volumineux
models/**/*.h5
models/**/*.bin
*.h5


### 4.4 Bonnes Pratiques Git

✅ **Commits atomiques** : Un commit = une fonctionnalité/fix

✅ **Messages descriptifs** : "Add BERT model with dropout regularization" (pas "fix bug")

✅ **Branches** : feature branches pour nouvelles fonctionnalités

✅ **Pull Requests** : Review de code avant merge

✅ **Tags** : Marquer les versions importantes (v1.0.0, v1.1.0, etc.)

✅ **Documentation** : README.md à jour avec instructions claires

## 5. Tests Unitaires <a id="5-tests"></a>

### 5.1 Pourquoi les Tests ?

Les tests unitaires garantissent :
- La **qualité** du code
- La **non-régression** lors de modifications
- La **confiance** lors du déploiement
- La **documentation** du comportement attendu

### 5.2 Structure des Tests

```
tests/
├── test_api.py              # Tests de l'API Flask
├── test_preprocessing.py    # Tests du prétraitement
└── test_model.py            # Tests du modèle
```

### 5.3 Exemple de Tests API

In [10]:
# Exemple de fichier tests/test_api.py
test_api_code = '''
import pytest
import json
from api.app import app

@pytest.fixture
def client():
    """Créer un client de test Flask"""
    app.config['TESTING'] = True
    with app.test_client() as client:
        yield client

def test_health_check(client):
    """Test du endpoint racine"""
    response = client.get('/')
    assert response.status_code == 200
    assert response.json['status'] == 'ok'

def test_predict_positive(client):
    """Test de prédiction avec tweet positif"""
    data = {"text": "I love this airline! Great service!"}
    response = client.post('/predict', 
                          data=json.dumps(data),
                          content_type='application/json')
    
    assert response.status_code == 200
    result = response.json
    assert 'sentiment' in result
    assert 'proba' in result
    assert result['sentiment'] in [0, 1]
    assert 0 <= result['proba'] <= 1

def test_predict_negative(client):
    """Test de prédiction avec tweet négatif"""
    data = {"text": "Terrible experience! Never flying again!"}
    response = client.post('/predict',
                          data=json.dumps(data),
                          content_type='application/json')
    
    assert response.status_code == 200
    result = response.json
    assert result['sentiment'] == 0  # Négatif

def test_predict_missing_text(client):
    """Test avec texte manquant"""
    response = client.post('/predict',
                          data=json.dumps({}),
                          content_type='application/json')
    
    assert response.status_code == 400
    assert 'error' in response.json

def test_predict_empty_text(client):
    """Test avec texte vide"""
    data = {"text": ""}
    response = client.post('/predict',
                          data=json.dumps(data),
                          content_type='application/json')
    
    assert response.status_code == 400

def test_batch_predict(client):
    """Test de prédiction batch"""
    data = {
        "texts": [
            "Great flight!",
            "Terrible service!",
            "Average experience"
        ]
    }
    response = client.post('/batch_predict',
                          data=json.dumps(data),
                          content_type='application/json')
    
    assert response.status_code == 200
    result = response.json
    assert 'predictions' in result
    assert len(result['predictions']) == 3
'''

print("Exemple de tests unitaires (tests/test_api.py) :")
print("="*80)
print(test_api_code)
print("="*80)

Exemple de tests unitaires (tests/test_api.py) :

import pytest
import json
from api.app import app

@pytest.fixture
def client():
    """Créer un client de test Flask"""
    app.config['TESTING'] = True
    with app.test_client() as client:
        yield client

def test_health_check(client):
    """Test du endpoint racine"""
    response = client.get('/')
    assert response.status_code == 200
    assert response.json['status'] == 'ok'

def test_predict_positive(client):
    """Test de prédiction avec tweet positif"""
    data = {"text": "I love this airline! Great service!"}
    response = client.post('/predict', 
                          data=json.dumps(data),
                          content_type='application/json')
    
    assert response.status_code == 200
    result = response.json
    assert 'sentiment' in result
    assert 'proba' in result
    assert result['sentiment'] in [0, 1]
    assert 0 <= result['proba'] <= 1

def test_predict_negative(client):
    """Test de prédi

### 5.4 Exécution des Tests

```bash
# Installer pytest
pip install pytest pytest-cov

# Lancer tous les tests
pytest tests/

# Lancer avec coverage
pytest --cov=api tests/

# Lancer avec rapport détaillé
pytest -v tests/
```

**Capture d'écran à inclure** : Résultat de l'exécution de `pytest -v tests/` montrant tous les tests qui passent

## 6. Pipeline CI/CD avec GitHub Actions <a id="6-cicd"></a>

### 6.1 Qu'est-ce que le CI/CD ?

**CI (Continuous Integration)** :
- Exécuter automatiquement les tests à chaque commit
- Vérifier la qualité du code (linting)
- Construire l'application

**CD (Continuous Deployment)** :
- Déployer automatiquement en production si tests OK
- Garantir une livraison rapide et fiable
- Réduire les erreurs humaines

### 6.2 Configuration GitHub Actions

Fichier `.github/workflows/deploy.yml` :

In [11]:
# Exemple de workflow GitHub Actions
workflow_yaml = '''
name: CI/CD Pipeline

on:
  push:
    branches: [ main ]
  pull_request:
    branches: [ main ]

jobs:
  test:
    runs-on: ubuntu-latest
    
    steps:
    - uses: actions/checkout@v2
    
    - name: Set up Python
      uses: actions/setup-python@v2
      with:
        python-version: 3.10
    
    - name: Install dependencies
      run: |
        python -m pip install --upgrade pip
        pip install -r api/requirements.txt
        pip install pytest pytest-cov
    
    - name: Run tests
      run: |
        pytest tests/ -v --cov=api
    
    - name: Upload coverage reports
      uses: codecov/codecov-action@v2
  
  build:
    needs: test
    runs-on: ubuntu-latest
    
    steps:
    - uses: actions/checkout@v2
    
    - name: Build Docker image
      run: |
        docker build -t airparadis-sentiment-api:latest ./api
    
    - name: Login to Docker Hub
      uses: docker/login-action@v1
      with:
        username: ${{ secrets.DOCKER_USERNAME }}
        password: ${{ secrets.DOCKER_PASSWORD }}
    
    - name: Push Docker image
      run: |
        docker tag airparadis-sentiment-api:latest ${{ secrets.DOCKER_USERNAME }}/airparadis-sentiment-api:latest
        docker push ${{ secrets.DOCKER_USERNAME }}/airparadis-sentiment-api:latest
  
  deploy:
    needs: build
    runs-on: ubuntu-latest
    if: github.ref == 'refs/heads/main'
    
    steps:
    - name: Deploy to Azure Web App
      uses: azure/webapps-deploy@v2
      with:
        app-name: airparadis-sentiment-api
        publish-profile: ${{ secrets.AZURE_WEBAPP_PUBLISH_PROFILE }}
        images: ${{ secrets.DOCKER_USERNAME }}/airparadis-sentiment-api:latest
'''

print("Fichier .github/workflows/deploy.yml :")
print("="*80)
print(workflow_yaml)
print("="*80)

Fichier .github/workflows/deploy.yml :

name: CI/CD Pipeline

on:
  push:
    branches: [ main ]
  pull_request:
    branches: [ main ]

jobs:
  test:
    runs-on: ubuntu-latest
    
    steps:
    - uses: actions/checkout@v2
    
    - name: Set up Python
      uses: actions/setup-python@v2
      with:
        python-version: 3.10
    
    - name: Install dependencies
      run: |
        python -m pip install --upgrade pip
        pip install -r api/requirements.txt
        pip install pytest pytest-cov
    
    - name: Run tests
      run: |
        pytest tests/ -v --cov=api
    
    - name: Upload coverage reports
      uses: codecov/codecov-action@v2
  
  build:
    needs: test
    runs-on: ubuntu-latest
    
    steps:
    - uses: actions/checkout@v2
    
    - name: Build Docker image
      run: |
        docker build -t airparadis-sentiment-api:latest ./api
    
    - name: Login to Docker Hub
      uses: docker/login-action@v1
      with:
        username: ${{ secrets.DOCKER_

### 6.3 Étapes du Pipeline

```
┌────────────────────────────────────────────────────────────┐
│  1. TRIGGER                                                │
│     Push sur main ou Pull Request                         │
└────────────────────────────────────────────────────────────┘
                         ↓
┌────────────────────────────────────────────────────────────┐
│  2. TEST                                                   │
│     - Checkout code                                        │
│     - Install Python 3.10                                  │
│     - Install dependencies                                 │
│     - Run pytest                                           │
└────────────────────────────────────────────────────────────┘
                         ↓
┌────────────────────────────────────────────────────────────┐
│  3. BUILD (si tests OK)                                    │
│     - Build Docker image                                   │
│     - Login to Docker Hub                                  │
│     - Push image to registry                               │
└────────────────────────────────────────────────────────────┘
                         ↓
┌────────────────────────────────────────────────────────────┐
│  4. DEPLOY (si build OK + branch main)                     │
│     - Deploy to Azure Web App                              │
│     - Update production environment                        │
└────────────────────────────────────────────────────────────┘
```

**Captures d'écran à inclure** :
- Vue de la liste des workflows GitHub Actions
- Détail d'un workflow réussi avec toutes les étapes
- Logs de l'exécution des tests

### 6.4 Avantages du CI/CD

✅ **Automatisation complète** : Plus besoin de déployer manuellement

✅ **Qualité garantie** : Déploiement seulement si tous les tests passent

✅ **Déploiement rapide** : De quelques minutes au lieu de plusieurs heures

✅ **Traçabilité** : Historique complet de tous les déploiements

✅ **Rollback facile** : Retour à une version précédente en un clic

✅ **Confiance** : Réduction des erreurs humaines

## 7. Monitoring en Production <a id="7-monitoring"></a>

### 7.1 Azure Application Insights

**Application Insights** permet de monitorer :
- Volume de requêtes
- Temps de réponse
- Taux d'erreur
- Traces personnalisées (prédictions non conformes)
- Exceptions et logs

### 7.2 Configuration dans l'API

In [12]:
# Exemple de configuration Application Insights dans Flask
app_insights_code = '''
from opencensus.ext.azure.log_exporter import AzureLogHandler
from opencensus.ext.flask.flask_middleware import FlaskMiddleware
from opencensus.ext.azure.trace_exporter import AzureExporter
import logging

# Configuration du logger
logger = logging.getLogger(__name__)
logger.addHandler(AzureLogHandler(
    connection_string='InstrumentationKey=YOUR_KEY'
))

# Ajouter le middleware Flask
middleware = FlaskMiddleware(
    app,
    exporter=AzureExporter(
        connection_string='InstrumentationKey=YOUR_KEY'
    ),
    sampler=ProbabilitySampler(rate=1.0)
)

# Dans le endpoint de prédiction
@app.route('/predict', methods=['POST'])
def predict():
    data = request.get_json()
    text = data.get('text')
    
    # Prédiction
    prediction = model.predict([text])[0]
    proba = model.predict_proba([text])[0][prediction]
    
    # Logger la prédiction
    logger.info(f"Prediction made: {prediction}, confidence: {proba:.2f}")
    
    # Si prédiction non conforme (faible confiance)
    if proba < 0.6:
        logger.warning(
            f"Low confidence prediction: {proba:.2f} for text: {text[:50]}",
            extra={'custom_dimensions': {
                'prediction': int(prediction),
                'confidence': float(proba),
                'text_length': len(text)
            }}
        )
    
    return jsonify({
        'sentiment': int(prediction),
        'proba': float(proba)
    })
'''

print("Exemple de configuration Application Insights :")
print("="*80)
print(app_insights_code)
print("="*80)

Exemple de configuration Application Insights :

from opencensus.ext.azure.log_exporter import AzureLogHandler
from opencensus.ext.flask.flask_middleware import FlaskMiddleware
from opencensus.ext.azure.trace_exporter import AzureExporter
import logging

# Configuration du logger
logger = logging.getLogger(__name__)
logger.addHandler(AzureLogHandler(
    connection_string='InstrumentationKey=YOUR_KEY'
))

# Ajouter le middleware Flask
middleware = FlaskMiddleware(
    app,
    exporter=AzureExporter(
        connection_string='InstrumentationKey=YOUR_KEY'
    ),
    sampler=ProbabilitySampler(rate=1.0)
)

# Dans le endpoint de prédiction
@app.route('/predict', methods=['POST'])
def predict():
    data = request.get_json()
    text = data.get('text')
    
    # Prédiction
    prediction = model.predict([text])[0]
    proba = model.predict_proba([text])[0][prediction]
    
    # Logger la prédiction
    logger.info(f"Prediction made: {prediction}, confidence: {proba:.2f}")
    
    # Si 

### 7.3 Alertes Configurées

Nous avons configuré plusieurs alertes dans Azure :

**1. Alerte de latence élevée**
- Condition : Temps de réponse > 500ms pendant 5 minutes
- Action : Email à l'équipe DevOps

**2. Alerte de taux d'erreur élevé**
- Condition : Erreurs > 5% des requêtes pendant 10 minutes
- Action : Email + SMS aux responsables

**3. Alerte de prédictions non conformes**
- Condition : > 20% de prédictions avec confiance < 60%
- Action : Email au Data Science team (re-training nécessaire)

**4. Alerte de volume anormal**
- Condition : Volume < 10 req/min (possible downtime)
- Action : SMS immédiat

### 7.4 Dashboard de Monitoring

**Captures d'écran à inclure** :
- Dashboard Application Insights avec graphiques de métriques
- Exemple de trace d'une prédiction non conforme
- Configuration d'une alerte
- Email/SMS d'alerte reçu

### 7.5 Stratégie de Maintenance du Modèle

#### Analyse de la Stabilité dans le Temps

**Indicateurs à surveiller** :
1. **Taux de prédictions non conformes** (confiance < 60%)
2. **Distribution des sentiments** (changement de ratio positif/négatif)
3. **Feedback utilisateurs** (corrections manuelles)
4. **Nouveaux mots/hashtags** non vus pendant l'entraînement

#### Actions d'Amélioration

**Court terme** (hebdomadaire) :
- Analyser les prédictions corrigées par les utilisateurs
- Identifier les patterns d'erreurs
- Enrichir le dataset avec ces exemples

**Moyen terme** (mensuel) :
- Re-entraîner le modèle avec nouvelles données
- Comparer performances ancien vs nouveau modèle
- Déployer si amélioration significative (> 2% F1-Score)

**Long terme** (trimestriel) :
- Analyser l'évolution du vocabulaire Twitter
- Tester de nouvelles architectures (nouveaux modèles BERT, etc.)
- Optimiser les hyperparamètres
- Évaluer le ROI du modèle (détections de bad buzz réussies)

#### Critères de Re-training

Déclencher un re-training si :
- ✅ > 1000 nouvelles corrections utilisateurs collectées
- ✅ Taux de prédictions non conformes > 25% (vs 15% initial)
- ✅ Changement significatif dans la distribution des données
- ✅ Nouveau vocabulaire détecté (nouveaux hashtags, événements)

#### Workflow de Re-training Automatisé

```
1. Collecte automatique des feedbacks (Streamlit → Database)
2. Détection de drift (monitoring Azure)
3. Trigger re-training (GitHub Actions scheduled workflow)
4. Entraînement et évaluation (MLFlow tracking)
5. Validation sur test set hold-out
6. Si amélioration > seuil → Deploy en Staging
7. A/B Testing Staging vs Production (1 semaine)
8. Si OK → Promote en Production
9. Ancien modèle → Archive dans MLFlow Registry
```

## 8. Conclusion <a id="8-conclusion"></a>

### Récapitulatif de la Démarche MLOps

Ce projet a mis en œuvre une démarche MLOps complète :

✅ **Tracking des expérimentations** : MLFlow pour suivre tous les runs

✅ **Stockage centralisé** : MLFlow Model Registry pour versionner les modèles

✅ **Versioning du code** : Git/GitHub avec historique complet

✅ **Tests automatisés** : pytest pour garantir la qualité

✅ **CI/CD** : GitHub Actions pour déploiement automatique

✅ **Monitoring** : Azure Application Insights pour surveillance production

✅ **Alertes** : Notifications en cas de problème

✅ **Stratégie de maintenance** : Plan de re-training et amélioration continue

### Bénéfices pour Air Paradis

**Qualité** :
- Tests automatisés → Moins de bugs
- Validation systématique avant déploiement

**Rapidité** :
- Déploiement en minutes (vs heures/jours)
- Itérations rapides avec feedback utilisateurs

**Fiabilité** :
- Monitoring 24/7 → Détection rapide des problèmes
- Rollback facile en cas d'incident

**Évolutivité** :
- Infrastructure Cloud scalable automatiquement
- Prêt pour croissance du volume de données

**Traçabilité** :
- Historique complet de tous les modèles
- Reproductibilité garantie

### Prochaines Étapes

1. **Enrichissement du dataset** avec tweets spécifiques Air Paradis
2. **A/B Testing** pour comparer modèles en production
3. **Optimisation des coûts** Cloud (caching, batching)
4. **Features avancées** : détection de sarcasme, analyse des émotions
5. **Dashboard métier** pour équipes marketing

---

**Ce notebook a démontré une mise en œuvre complète et professionnelle des pratiques MLOps, conformément aux exigences du projet.**